# Hồi quy logistic với hiệu chỉnh Ridge và Lasso

*Bài tập này là phần tiếp theo của bài ex_02, tôi sẽ không giải thích lại về các khái niệm PCA, TPR, FPR, ROC curve, auc ... cũng như không chạy lại mô hình ở bài ex_02 mà sử dụng luôn kết quả để so sánh. Mô hình ở bài ex_02 đã được lưu lại trong folder model để có thể tái sử dụng.*



In [41]:
import pandas as pd
import matplotlib.pyplot as plt

Load dữ liệu bằng thư viện pandas

In [42]:
df_raw = pd.read_csv('../data/sonar_zip/data/sonar_csv.csv')

In [43]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   attribute_1   208 non-null    float64
 1   attribute_2   208 non-null    float64
 2   attribute_3   208 non-null    float64
 3   attribute_4   208 non-null    float64
 4   attribute_5   208 non-null    float64
 5   attribute_6   208 non-null    float64
 6   attribute_7   208 non-null    float64
 7   attribute_8   208 non-null    float64
 8   attribute_9   208 non-null    float64
 9   attribute_10  208 non-null    float64
 10  attribute_11  208 non-null    float64
 11  attribute_12  208 non-null    float64
 12  attribute_13  208 non-null    float64
 13  attribute_14  208 non-null    float64
 14  attribute_15  208 non-null    float64
 15  attribute_16  208 non-null    float64
 16  attribute_17  208 non-null    float64
 17  attribute_18  208 non-null    float64
 18  attribute_19  208 non-null    

In [44]:
df_raw.head()

,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,attribute_52,attribute_53,attribute_54,attribute_55,attribute_56,attribute_57,attribute_58,attribute_59,attribute_60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,Rock
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,Rock
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,Rock
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,Rock
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,Rock


Describe dataset 

In [45]:
df_raw.describe()

,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,attribute_51,attribute_52,attribute_53,attribute_54,attribute_55,attribute_56,attribute_57,attribute_58,attribute_59,attribute_60
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [46]:
df_attr_raw = df_raw.loc[:, df_raw.columns != 'Class']
df_cls = df_raw['Class']

In [47]:
df_cls.describe()

count      208
unique       2
top       Mine
freq       111
Name: Class, dtype: object

In [48]:
df_cls.head()
df_attr_raw.head()

,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,attribute_51,attribute_52,attribute_53,attribute_54,attribute_55,attribute_56,attribute_57,attribute_58,attribute_59,attribute_60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [49]:
df_attr = (df_attr_raw - df_attr_raw.min()) / (df_attr_raw.max() - df_attr_raw.min())

In [50]:
df_attr.describe()

,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,attribute_6,attribute_7,attribute_8,attribute_9,attribute_10,...,attribute_51,attribute_52,attribute_53,attribute_54,attribute_55,attribute_56,attribute_57,attribute_58,attribute_59,attribute_60
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.204011,0.162180,0.139068,0.114342,0.173732,0.253615,0.320472,0.285114,0.252485,0.281652,...,0.160047,0.180031,0.265172,0.290669,0.197061,0.200555,0.213642,0.175035,0.216015,0.136425
std,0.169550,0.141277,0.126242,0.110623,0.140888,0.158843,0.167175,0.187767,0.175311,0.192215,...,0.119607,0.137432,0.183385,0.213474,0.160717,0.147080,0.164361,0.148051,0.170286,0.116190
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.087389,0.067938,0.057326,0.044163,0.079508,0.152714,0.209957,0.165215,0.132571,0.142964,...,0.083914,0.092368,0.118831,0.127924,0.080499,0.102564,0.096591,0.075515,0.098485,0.057737
50%,0.157080,0.129447,0.107753,0.090942,0.141517,0.220236,0.280438,0.235061,0.214349,0.244673,...,0.138446,0.151213,0.235065,0.242690,0.156463,0.165385,0.160511,0.125858,0.173554,0.108545
75%,0.251106,0.202958,0.185447,0.139563,0.237319,0.333042,0.407738,0.361852,0.334555,0.368082,...,0.207420,0.227175,0.374026,0.394737,0.260771,0.260897,0.287642,0.229977,0.281680,0.183025
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Mã hóa nhãn về dạng 1, 0

In [51]:
def tmp(r):
  if r == 'Rock':
    return 1
  return 0

df_cls = df_raw['Class'].apply(lambda row : tmp(row))

In [52]:
df_cls.head()

0    1
1    1
2    1
3    1
4    1
Name: Class, dtype: int64

Thử chia dữ liệu train, test với random_state bất kì, tỉ lệ mặc định theo sklearn là 25% dữ liệu cho tập test

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_attr, df_cls, random_state=42)

Mặc định thì thư viện sklearn đã có hiệu chỉnh L2 và tự tìm tham số hiệu chỉnh tốt nhất. 

In [54]:
from sklearn.linear_model import LogisticRegression
logreg_l2 = LogisticRegression(random_state=42, solver='saga')
model_l2 = logreg_l2.fit(X_train, y_train)

/home/linhnm/msc_code/social_stats/venv/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Giá trị accuracy và auc của ROC cho hiệu chỉnh L2

In [55]:
from sklearn.metrics import roc_curve, auc
acc_l2 = model_l2.score(X_test, y_test)
y_pred_prob_l2 = model_l2.predict_proba(X_test)[:,1]
fpr_l2, tpr_l2, thresholds_l2 = roc_curve(y_test, y_pred_prob_l2)
auc_roc_l2 = auc(fpr_l2, tpr_l2)

print(f'acc: {acc_l2}, auc: {auc_roc_l2}')

acc: 0.8653846153846154, auc: 0.9378787878787879


L1 regularization

In [56]:
from sklearn.linear_model import LogisticRegression
logreg_l1 = LogisticRegression(random_state=42, penalty='l1', solver='saga', max_iter=1000)
model_l1 = logreg_l1.fit(X_train, y_train)

Giá trị accuracy và auc của ROC cho hiệu chỉnh L1. Với bộ tham số cấu hình như thế này thì hiệu chỉnh L2 (Ridge) đang thể hiện tốt hơn hiệu chỉnh L1 (Lasso).

In [57]:
from sklearn.metrics import roc_curve, auc
acc_l1 = model_l1.score(X_test, y_test)
y_pred_prob_l1 = model_l1.predict_proba(X_test)[:,1]
fpr_l1, tpr_l1, thresholds_l1 = roc_curve(y_test, y_pred_prob_l1)
auc_roc_l1 = auc(fpr_l1, tpr_l1)

print(f'acc: {acc_l1}, auc: {auc_roc_l1}')

acc: 0.8461538461538461, auc: 0.9287878787878788


Tổ hợp của hiệu chỉnh L1 và L2 được gọi là hiệu chỉnh elasticnet. Chọn tham số l1_ratio = 0.5 nghĩa là tham số hiệu chỉnh L1 và L2 là như nhau.
Gọi $f(w)$ là loss function. Hiệu chỉnh có dạng 
$$g(w) = f(w) + \alpha \| w\|_1 + \beta \| w\|_2^2$$
$$\text{l1\_ratio} = \frac{\alpha}{\alpha + \beta}$$

In [58]:
logreg_els = LogisticRegression(random_state=42, penalty='elasticnet', l1_ratio=0.5, solver='saga', max_iter=1000)
model_els = logreg_els.fit(X_train, y_train)

In [59]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
acc_els = model_els.score(X_test, y_test)
y_pred_prob_els = model_els.predict_proba(X_test)[:,1]
fpr_els, tpr_els, thresholds_els = roc_curve(y_test, y_pred_prob_els)
auc_roc_els = auc(fpr_els, tpr_els)

print(f'acc: {acc_els}, auc: {auc_roc_els}')

acc: 0.8653846153846154, auc: 0.9348484848484848


Chúng ta đi tìm mô hình tốt nhất bằng cách chia tập train và test theo nhiều cách khác nhau. Có thể sử dụng k-fold nhưng nhìn chung với mỗi cách chia tập train, test, ta sẽ có một mô hình khác nhau và từ đó tìm ra mô hình tốt nhất hay là mô hình có chỉ số AUC của ROC curve là cao nhất.

In [60]:
def scan_random_state(df_features, df_labels, N, penalty, l1_ratio, solver, max_iter):
  X_train = list()
  X_test = list()
  y_train = list()
  y_test = list()
  model = list()
  auc_model = list()
  for i in range(N):
    _X_train, _X_test, _y_train, _y_test = train_test_split(df_features, df_labels, random_state=i)
    X_train.append(_X_train)
    X_test.append(_X_test)
    y_train.append(_y_train)
    y_test.append(_y_test)
    
    logreg = LogisticRegression(random_state=i, penalty=penalty, l1_ratio=l1_ratio, solver=solver, max_iter=max_iter)
    _model = logreg.fit(_X_train, _y_train)
    model.append(_model)
    _y_pred_prob = _model.predict_proba(_X_test)[:,1]
    
    auc_model.append(roc_auc_score(_y_test, _y_pred_prob))
    
  return X_train, X_test, y_train, y_test, model, auc_model
  

Thử với 1000 cách chia tập train, test với tỉ lệ mặc định 0.25 cho tập test, lấy ra mô hình có AUC cao nhất. Giá trị tìm được là 0.979, tốt hơn so với mô hình trước đó.

In [61]:
# logreg_l1 = LogisticRegression(random_state=42, penalty='l1', solver='saga', max_iter=1000)
# logreg_l2 = LogisticRegression(random_state=42, penalty='l2', solver='saga', max_iter=1000)
# logreg_els = LogisticRegression(random_state=42, penalty='elasticnet', l1_ratio=0.5, solver='saga', max_iter=1000)

In [62]:
X_train_l1, X_test_l1, y_train_l1, y_test_l1, model_l1, auc_model_l1 = scan_random_state(df_attr, df_cls, 1000, 'l1', None, 'saga', 1000)

In [63]:
# auc_model

max_auc = max(auc_model_l1)
best_random_state = auc_model_l1.index(max_auc)

print(f'best random state: {best_random_state} with auc: {max_auc}')

best random state: 889 with auc: 0.9792592592592593


In [66]:
X_train_els, X_test_els, y_train_els, y_test_els, model_els, auc_model_els = scan_random_state(df_attr, df_cls, 1000, 'elasticnet', 0.5, 'saga', 1000)

In [67]:
# auc_model

max_auc = max(auc_model_els)
best_random_state = auc_model_els.index(max_auc)

print(f'best random state: {best_random_state} with auc: {max_auc}')

best random state: 889 with auc: 0.9807407407407407


Ta có thể lặp lại các tính toán trên với việc phân tích thành phần chính (PCA) cho tập dữ liệu sonar

Trước hết ta chọn ngẫu nhiên số chiều dữ liệu, ở đây tôi chọn là 30 trên 60 và thử tính toán như trên.

In [68]:
from sklearn.decomposition import PCA
pca = PCA(n_components=30)
df_attr_pca = df_attr
df_attr_pca = pca.fit_transform(df_attr_pca)

In [69]:
df_attr_pca.shape
df_attr_pca = pd.DataFrame(df_attr_pca)

In [70]:
df_attr_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21      208 non-null    float64
 22  22

In [71]:
X_train, X_test, y_train, y_test = train_test_split(df_attr_pca, df_cls, random_state=42)

Thử scan với 1000 random_state khác nhau ta được auc tốt nhất là 0.971, tốt hơn khi không dùng PCA.

In [72]:
X_train, X_test, y_train, y_test, model, auc_model = scan_random_state(df_attr_pca, df_cls, 1000, 'l1', None, 'saga', 1000)

In [73]:
max_auc = max(auc_model)
best_random_state = auc_model.index(max_auc)

print(f'best random state: {best_random_state} with auc: {max_auc}')

score = model[best_random_state].score(X_test[best_random_state], y_test[best_random_state])
print(f'best random state: {best_random_state} with accuracy: {score}')

best random state: 889 with auc: 0.9718518518518519
best random state: 889 with accuracy: 0.9615384615384616


In [74]:
def scan_num_feature(min_num_feature, max_num_feature, N_rand_state, penalty, l1_ratio, solver, max_iter):
  X_train_dict = dict()
  X_test_dict = dict()
  y_train_dict = dict()
  y_test_dict = dict()
  model_dict = dict()
  auc_model_dict = dict()
  best_random_state_dict = dict()
  
  for i in range(min_num_feature, max_num_feature):
    pca = PCA(n_components=i)
    df_attr_pca = df_attr
    df_attr_pca = pca.fit_transform(df_attr_pca)
    df_attr_pca = pd.DataFrame(df_attr_pca)
    X_train, X_test, y_train, y_test, model, auc_model = scan_random_state(df_attr_pca, df_cls, N_rand_state, penalty, l1_ratio, solver, max_iter)
    
    max_auc = max(auc_model)
    best_random_state = auc_model.index(max_auc)
    
    X_train_dict[i] = X_train[best_random_state]
    X_test_dict[i] = X_test[best_random_state]
    y_train_dict[i] = y_train[best_random_state]
    y_test_dict[i] = y_test[best_random_state]
    model_dict[i] = model[best_random_state]
    auc_model_dict[i] = auc_model[best_random_state]
    best_random_state_dict[i] = best_random_state

    print(f'number of features: {i}, best random state: {best_random_state} with auc: {max_auc}')

    score = model[best_random_state].score(X_test[best_random_state], y_test[best_random_state])
    print(f'number of features: {i}, best random state: {best_random_state} with accuracy: {score}')
    
  return X_train_dict, X_test_dict, y_train_dict, y_test_dict, model_dict, auc_model_dict, best_random_state_dict

In [75]:
X_train_dict, X_test_dict, y_train_dict, y_test_dict, model_dict, auc_model_dict, best_random_state_dict = scan_num_feature(1, 60, 1000, 'l1', None, 'saga', 1000)

number of features: 1, best random state: 913 with auc: 0.7574404761904763
number of features: 1, best random state: 913 with accuracy: 0.5384615384615384
number of features: 2, best random state: 604 with auc: 0.8245877061469264
number of features: 2, best random state: 604 with accuracy: 0.5576923076923077
number of features: 3, best random state: 442 with auc: 0.9636363636363636
number of features: 3, best random state: 442 with accuracy: 0.8653846153846154
number of features: 4, best random state: 285 with auc: 0.96
number of features: 4, best random state: 285 with accuracy: 0.9423076923076923
number of features: 5, best random state: 285 with auc: 0.9481481481481482
number of features: 5, best random state: 285 with accuracy: 0.8846153846153846
number of features: 6, best random state: 889 with auc: 0.9540740740740741
number of features: 6, best random state: 889 with accuracy: 0.8653846153846154
number of features: 7, best random state: 889 with auc: 0.9733333333333334
number of

In [76]:
best_num_feature = [key for key, value in auc_model_dict.items() if value == max(auc_model_dict.values())]

j = best_num_feature[0]

print(f'number of features with best auc: {best_num_feature}')
print(f'best auc: {auc_model_dict[j]}')
print(f'accuracy: {model_dict[j].score(X_test_dict[j], y_test_dict[j])}')
print(f'best random state: {best_random_state_dict[j]}')

number of features with best auc: [15, 16, 17, 18, 19]
best auc: 0.9762962962962963
accuracy: 0.9615384615384616
best random state: 889


Ta thấy rằng thực hiện phân tích thành phần chính với 15 features cho ra kết quả tốt nhất với auc = 0.976. So với hiệu chỉnh L2 với auc = 0.994. Như vậy hiệu chỉnh L2 là tốt hơn so với hiệu chỉnh L1.
Tuy nhiên hiệu chỉnh L2 dùng 22 features chính, nặng hơn so với 15 features chính của hiệu chỉnh L1.

In [79]:
import pickle
file_name = '../model/pca_data_best_model_l1.sav'
pickle.dump(model_dict[j], open(file_name, 'wb'))

In [80]:
y_pred = model_dict[j].predict(X_test_dict[j])

In [81]:
from sklearn.metrics import classification_report
# confusion matrix
print(classification_report(y_test_dict[j], y_pred))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        25
           1       0.93      1.00      0.96        27

    accuracy                           0.96        52
   macro avg       0.97      0.96      0.96        52
weighted avg       0.96      0.96      0.96        52



Sử dụng hiệu chỉnh elasticnet, ta cần điều chỉnh tham số l1_ratio để tìm được mô hình tốt nhất. Tuy nhiên việc scan l1_ratio cùng với số features chính là khá tốn thời gian. Vì vậy, tôi lựa chọn tham số l1_ratio=0.5 và scan số features chính tương ứng.

In [82]:
X_train_dict, X_test_dict, y_train_dict, y_test_dict, model_dict, auc_model_dict, best_random_state_dict = scan_num_feature(1, 60, 1000, 'elasticnet', 0.5, 'saga', 1000)

number of features: 1, best random state: 541 with auc: 0.794074074074074
number of features: 1, best random state: 541 with accuracy: 0.5192307692307693
number of features: 2, best random state: 604 with auc: 0.8425787106446777
number of features: 2, best random state: 604 with accuracy: 0.5576923076923077
number of features: 3, best random state: 442 with auc: 0.9681818181818183
number of features: 3, best random state: 442 with accuracy: 0.8653846153846154
number of features: 4, best random state: 285 with auc: 0.9614814814814815
number of features: 4, best random state: 285 with accuracy: 0.9038461538461539
number of features: 5, best random state: 889 with auc: 0.9451851851851851
number of features: 5, best random state: 889 with accuracy: 0.8653846153846154
number of features: 6, best random state: 889 with auc: 0.9511111111111111
number of features: 6, best random state: 889 with accuracy: 0.8846153846153846
number of features: 7, best random state: 889 with auc: 0.9777777777777

auc cho ra là 0.991, có tốt hơn so với chỉ dùng hiệu chỉnh L1, tuy nhiên chưa tốt bằng chỉ sử dụng L2. Một điểm thú vị là số features chính có thể dùng là từ 22 đến 59 hay có rất nhiều lựa chọn ở đây. 

In [83]:
best_num_feature = [key for key, value in auc_model_dict.items() if value == max(auc_model_dict.values())]

j = best_num_feature[0]

print(f'number of features with best auc: {best_num_feature}')
print(f'best auc: {auc_model_dict[j]}')
print(f'accuracy: {model_dict[j].score(X_test_dict[j], y_test_dict[j])}')
print(f'best random state: {best_random_state_dict[j]}')

number of features with best auc: [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
best auc: 0.9911111111111112
accuracy: 0.9615384615384616
best random state: 889


In [84]:
X_train_dict, X_test_dict, y_train_dict, y_test_dict, model_dict, auc_model_dict, best_random_state_dict = scan_num_feature(1, 60, 1000, 'elasticnet', 0.1, 'saga', 1000)

number of features: 1, best random state: 541 with auc: 0.794074074074074
number of features: 1, best random state: 541 with accuracy: 0.5192307692307693
number of features: 2, best random state: 604 with auc: 0.848575712143928
number of features: 2, best random state: 604 with accuracy: 0.5769230769230769
number of features: 3, best random state: 442 with auc: 0.9712121212121212
number of features: 3, best random state: 442 with accuracy: 0.8846153846153846
number of features: 4, best random state: 285 with auc: 0.9614814814814815
number of features: 4, best random state: 285 with accuracy: 0.9038461538461539
number of features: 5, best random state: 889 with auc: 0.9422222222222222
number of features: 5, best random state: 889 with accuracy: 0.8846153846153846
number of features: 6, best random state: 889 with auc: 0.9525925925925925
number of features: 6, best random state: 889 with accuracy: 0.8846153846153846
number of features: 7, best random state: 889 with auc: 0.97629629629629

Khi giảm trọng số của hiệu chỉnh L1 (về 0.1), mô hình có auc tốt hơn là auc = 0.994. 

In [85]:
best_num_feature = [key for key, value in auc_model_dict.items() if value == max(auc_model_dict.values())]

j = best_num_feature[0]

print(f'number of features with best auc: {best_num_feature}')
print(f'best auc: {auc_model_dict[j]}')
print(f'accuracy: {model_dict[j].score(X_test_dict[j], y_test_dict[j])}')
print(f'best random state: {best_random_state_dict[j]}')

number of features with best auc: [22, 23, 24, 34]
best auc: 0.9940740740740741
accuracy: 0.9230769230769231
best random state: 889


Như vậy mô hình tốt nhất ta có thể tìm được chính là mô hình trong bài tập ex_02 với hiệu chỉnh L2, PCA với 22 thành phần chính, random_state=889 cho auc=0.994.
Mô hình đơn giản hơn sử dụng 15 thành phần chính với hiệu chỉnh L1 cho ra auc=0.976.
Tuy sử dụng ít thành phần chính hơn nhưng hiệu chỉnh L1 lại tốn thời gian hơn để huận luyện do bản chất chuẩn L1 không có đạo hàm liên tục, các thuật toán tối ưu sẽ giải chậm hơn so với trường hợp hiệu chỉnh L2 (có đạo hàm liên tục).